In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.preprocessing import scale

# Нужно подобрать оптимальное значение k для алгоритма kNN.

Будем использовать набор данных Wine, где требуется
предсказать сорт винограда, из которого изготовлено вино, используя
результаты химических анализов.
Выполните следующие шаги:
    
1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machinelearning-databases/wine/wine.data
2. Извлеките из данных признаки и классы. Класс записан в первом
столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл
wine.names, приложенный к заданию)


In [2]:
wine_features = pd.read_csv('wine.data')

wine_class = wine_features.pop('1')
wine_class

wine_features

,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065
0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
2,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
3,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
4,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
173,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
174,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
175,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


3. Оценку качества необходимо провести методом кросс-валидации по
5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для
воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).

4. Найдите точность классификации на кросс-валидации для метода
k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при
k от 1 до 50. При каком k получилось оптимальное качество? Чему
оно равно (число в интервале от 0 до 1)? Данные результаты и
будут ответами на вопросы 1 и 2.


In [3]:
clf_knn = KNeighborsClassifier()

In [4]:
search = GridSearchCV(clf_knn, {'n_neighbors': range(1, 51)}, cv=KFold(n_splits=5, shuffle=True, random_state=42))

In [5]:
search.fit(wine_features, wine_class)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(1, 51)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [6]:
search.best_params_ # Верно, и это скорее всего означает что метод плохо работает на имеющихся признаках.

{'n_neighbors': 1}

In [7]:
search.best_score_ 
# Метод угадывает правильный ответ в более чем половине случаев. Можно ли получить большее качество за счет 
# приведения признаков к одному масштабу?

0.7352380952380952

5. Произведите масштабирование признаков с помощью функции
sklearn.preprocessing.scale. Снова найдите оптимальное k на кроссвалидации.
6. Какое значение k получилось оптимальным после приведения признаков к одному масштабу? Как изменилось значение качества?
Приведите ответы на вопросы 3 и 4.

Приведение признаков к одному масштабу можно делать с помощью функции sklearn.preprocessing.scale, которой на вход необходимо подать матрицу признаков и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение и единичное стандартное отклонение.

In [8]:
wine_features = pd.DataFrame(scale(wine_features))
wine_features

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.255824,-0.501624,-0.824485,-2.500110,0.029180,0.572666,0.739636,-0.823139,-0.538575,-0.291130,0.407100,1.131698,0.971052
1,0.206229,0.018020,1.107690,-0.275639,0.099932,0.812784,1.221645,-0.501337,2.145975,0.269729,0.319567,0.804579,1.400998
2,1.706501,-0.349315,0.487935,-0.816726,0.948953,2.493609,1.472691,-0.984039,1.040572,1.184362,-0.424458,1.202811,2.340509
3,0.305420,0.224086,1.836812,0.445811,1.302712,0.812784,0.669343,0.222716,0.408913,-0.317016,0.363334,0.463238,-0.032154
4,1.495719,-0.519543,0.305655,-1.297693,0.878201,1.565153,1.372273,-0.179536,0.672104,0.731360,0.407100,0.349457,2.244965
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,0.888171,2.965658,0.305655,0.295509,-0.324579,-0.980097,-1.419361,1.268570,-0.924589,1.141219,-1.387314,-1.229247,-0.016230
173,0.503803,1.406725,0.415023,1.047020,0.170684,-0.788003,-1.278776,0.544517,-0.310476,0.968647,-1.124717,-1.485253,0.015618
174,0.342617,1.738222,-0.387012,0.145207,1.444215,-1.124168,-1.339027,0.544517,-0.415753,2.219795,-1.606145,-1.485253,0.286324
175,0.218628,0.224086,0.014006,0.145207,1.444215,-1.028121,-1.349068,1.349020,-0.222746,1.831507,-1.562379,-1.399918,0.302248


In [9]:
search = GridSearchCV(clf_knn, {'n_neighbors': range(1, 51)}, cv=KFold(n_splits=5, shuffle=True, random_state=42))

In [10]:
search.fit(wine_features, wine_class)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(1, 51)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=0)

In [11]:
search.best_params_ # Оптимальное качество достигается при k > 1. Это гораздо более логичный результат, чем до масштабирования.

{'n_neighbors': 28}

In [12]:
search.score(wine_features, wine_class) 
# Согласитесь, результат существенно превосходит качество, полученное без масштабирования признаков.

0.9830508474576272

# Следующая задача:
Мы будем использовать в данном задании набор данных Boston, где нужно предсказать стоимость жилья на основе различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). Подробнее о признаках можно почитать по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/housing/

Нам понадобится решать задачу регрессии с помощью метода k ближайших соседей — воспользуйтесь для этого классом sklearn.neighbors.KNeighborsRegressor. Метрика задается с помощью параметра metric, нас будет интересовать значение ’minkowski’. Параметр метрики Минковского задается с помощью параметра p данного класса.

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского:

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/IVbNGtGfEeWLWxKOx1I7mw_6e7648048323d85395ddb630fdd95a7c_mink.png?expiry=1588032000000&hmac=478LBmvXdtc6JynIZGKkQMa8CS7W2PQbOMtkctshlj8" width=400px>

Параметром метрики Минковского является число p, которое мы и будем настраивать.

1. Загрузите выборку Boston с помощью функции sklearn.datasets.load_boston(). Результатом вызова данной функции является объект, у которого признаки записаны в поле data, а целевой вектор — в поле target.
2. Приведите признаки в выборке к одному масштабу при помощи функции sklearn.preprocessing.scale.

In [13]:
from sklearn.datasets import load_boston
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [14]:
boston_data = load_boston() # np array
boston_features = pd.DataFrame(boston_data.data, columns=boston_data.feature_names) # to DataFrame
boston_target =  pd.DataFrame(boston_data.target) # to DataFrame
boston_features

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [15]:
boston_features = scale(boston_features)
boston_features

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

3. Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (используйте функцию numpy.linspace). Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки scikit-learn версии 0.18.1 и выше необходимо указывать scoring='neg_mean_squared_error'). Качество оценивайте, как и в предыдущем задании, с помощью кросс-валидации по 5 блокам с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).
4. Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей. Это значение параметра и будет ответом на задачу.

In [16]:
clf_regknn = KNeighborsRegressor(metric='minkowski', n_neighbors=5, weights='distance')

In [17]:
search = GridSearchCV(clf_regknn, 
                      {'p': np.linspace(1, 10, 200)}, 
                      cv=KFold(n_splits=5, 
                               shuffle=True, 
                               random_state=42), 
                      scoring='neg_mean_squared_error')

In [18]:
search.fit(boston_features, boston_target)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='distance'),
             iid='deprecated', n_jobs=None,
             param_grid={'p': array([ 1.        ,  1.04522613,  1.09045226,  1.13567839,  1.18090452,
        1.22613065,  1.27135678,  1....
        8.91457286,  8.95979899,  9.00502513,  9.05025126,  9.09547739,
        9.14070352,  9.18592965,  9.23115578,  9.27638191,  9.32160804,
        9.36683417,  9.4120603 ,  9.45728643,  9.50251256,  9.54773869,
        9.59296482,  9.63819095,  9.68341709,  9.72864322,  9.77386935,
        9.81909548,  9.86432161,  9.90954774,  9.95477387, 10.        ])},
             p

In [19]:
search.best_score_

-16.030646734221644

In [20]:
search.best_params_

{'p': 1.0}

Обратите внимание, что евклидова метрика (p=2) — не самый лучший вариант для этой задачи!

# Следующая задача:
## Введение
Линейные алгоритмы — распространенный класс моделей, которые отличается своей простотой и скоростью работы. Их можно обучать за разумное время на очень больших объемах данных, и при этом они могут работать с любыми типами признаков — вещественными, категориальными, разреженными. В этом задании мы предлагаем вам воспользоваться персептроном — одним из простейших вариантов линейных моделей.

Как и в случае с метрическими методами, качество линейных алгоритмов зависит от некоторых свойств данных. В частности, признаки должны быть нормализованы, то есть иметь одинаковый масштаб. Если это не так, и масштаб одного признака сильно превосходит масштаб других, то качество может резко упасть.

Один из способов нормализации заключается в стандартизации признаков. Для этого берется набор значений признака на всех объектах, вычисляется их среднее значение и стандартное отклонение. После этого из всех значений признака вычитается среднее, и затем полученная разность делится на стандартное отклонение.

Для стандартизации признаков удобно воспользоваться классом sklearn.preprocessing.StandardScaler. Функция fit_transform данного класса находит параметры нормализации (средние и дисперсии каждого признака) по выборке, и сразу же делает нормализацию выборки с использованием этих параметров. Функция transform делает нормализацию на основе уже найденных параметров.

In [21]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
# первым аргументом является вектор правильных ответов, а вторым — вектор ответов алгоритма.
from sklearn.preprocessing import StandardScaler

1. Загрузите обучающую и тестовую выборки из файлов perceptron-train.csv и perceptron-test.csv. Целевая переменная записана в первом столбце, признаки — во втором и третьем.
2. Обучите персептрон со стандартными параметрами и random_state=241.

In [22]:
perceptron_train = pd.read_csv('_3abd237d917280ba0d83bfe6bd49776f_perceptron-train.csv', 
                               header=None, 
                               names=['target', 'feature_1', 'feature_2'])

y_train = perceptron_train.pop('target')
y_train

0     -1.0
1      1.0
2     -1.0
3     -1.0
4      1.0
      ... 
295   -1.0
296   -1.0
297    1.0
298    1.0
299    1.0
Name: target, Length: 300, dtype: float64

In [23]:
perceptron_test = pd.read_csv('_3abd237d917280ba0d83bfe6bd49776f_perceptron-test.csv', 
                              header=None, 
                              names=['target', 'feature_1', 'feature_2'])

y_test = perceptron_test.pop('target')
y_test

0     -1.0
1      1.0
2     -1.0
3     -1.0
4      1.0
      ... 
195   -1.0
196    1.0
197    1.0
198   -1.0
199    1.0
Name: target, Length: 200, dtype: float64

In [24]:
prcptrn = Perceptron(random_state=241, max_iter=5, tol=None)

In [25]:
prcptrn.fit(perceptron_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=241, shuffle=True, tol=None,
           validation_fraction=0.1, verbose=0, warm_start=False)

3. Подсчитайте качество (долю правильно классифицированных объектов, accuracy) полученного классификатора на тестовой выборке.

In [26]:
prcptrn.score(perceptron_test, y_test)

0.655

4. Нормализуйте обучающую и тестовую выборку с помощью класса StandardScaler.

In [27]:
std_scaler = StandardScaler()
std_scaler.fit(perceptron_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [28]:
perceptron_train = std_scaler.fit_transform(perceptron_train)
perceptron_test = std_scaler.transform(perceptron_test)

perceptron_test

array([[ 1.67171578e+00,  1.50744899e+00],
       [-8.32374159e-01,  1.35261123e+00],
       [ 8.14843791e-01, -4.12493370e-01],
       [ 2.07920119e-01,  2.16572204e+00],
       [-4.02605851e-01,  6.93187424e-01],
       [-1.46865651e+00,  1.07554509e+00],
       [-1.21763005e+00, -3.83631116e-01],
       [-3.61791038e-01, -1.08341175e+00],
       [ 5.01097343e-01, -6.69052911e-02],
       [-8.48618151e-01, -1.80841461e+00],
       [ 7.05690653e-01,  4.93807275e-01],
       [ 1.08149150e-01,  1.83716080e+00],
       [ 5.80169128e-01,  4.84600472e-01],
       [ 7.79082214e-01,  3.87322485e+00],
       [-6.88592246e-02,  1.03931536e+00],
       [ 1.26539481e+00,  1.78114706e+00],
       [-3.59393234e-01, -4.09192508e-01],
       [-8.31317070e-01,  5.06364987e-01],
       [-9.84692062e-01,  9.19730255e-01],
       [-3.23600125e-01, -6.09350961e-01],
       [ 2.97171869e+00, -1.24493950e+00],
       [ 4.62837305e-01, -8.65868113e-02],
       [-3.47734322e-01, -1.25743507e+00],
       [ 1.

5. Обучите персептрон на новой выборке. Найдите долю правильных ответов на тестовой выборке.

In [29]:
prcptrn.fit(perceptron_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=241, shuffle=True, tol=None,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [30]:
prcptrn.score(perceptron_test, y_test)

0.84

6. Найдите разность между качеством на тестовой выборке после нормализации и качеством до нее. Это число и будет ответом на задание.

In [31]:
0.84 - 0.655

0.18499999999999994